In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Try the model with top 10 pharm companies - calculate asset/stock returns

In [38]:
PFEpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/PFEdata.csv'
PFEPrice = pd.read_csv(PFEpath)
PFEPrice = PFEPrice.set_index(pd.DatetimeIndex(PFEPrice['Date'].values))
PFEPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
PFEPrice = PFEPrice.rename(columns={"Adj Close": "PFE"})
PFEPrice

,PFE
2000-01-03,13.061072
2000-01-04,12.574479
2000-01-05,12.779366
2000-01-06,13.240340
2000-01-07,14.136689
...,...
2024-02-12,27.660000
2024-02-13,26.969999
2024-02-14,27.110001
2024-02-15,27.510000


In [24]:
GSKpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/GSKdata.csv'
GSKPrice = pd.read_csv(GSKpath)
GSKPrice = GSKPrice.set_index(pd.DatetimeIndex(GSKPrice['Date'].values))
GSKPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
GSKPrice = GSKPrice.rename(columns={"Adj Close": "GSK"})
GSKPrice

,GSK
2000-01-03,18.336874
2000-01-04,17.862465
2000-01-05,18.460629
2000-01-06,18.192493
2000-01-07,19.141298
...,...
2024-02-12,40.672421
2024-02-13,40.989323
2024-02-14,41.395355
2024-02-15,41.365646


In [26]:
SNYpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/SNYdata.csv'
SNYPrice = pd.read_csv(SNYpath)
SNYPrice = SNYPrice.set_index(pd.DatetimeIndex(SNYPrice['Date'].values))
SNYPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
SNYPrice = SNYPrice.rename(columns={"Adj Close": "SNY"})
SNYPrice

,SNY
2002-07-01,14.027581
2002-07-02,13.594075
2002-07-03,13.476276
2002-07-05,14.324433
2002-07-08,14.724953
...,...
2024-02-12,46.340000
2024-02-13,45.750000
2024-02-14,45.799999
2024-02-15,46.389999


In [27]:
AZNpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/AZNdata.csv'
AZNPrice = pd.read_csv(AZNpath)
AZNPrice = AZNPrice.set_index(pd.DatetimeIndex(AZNPrice['Date'].values))
AZNPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
AZNPrice = AZNPrice.rename(columns={"Adj Close": "AZN"})
AZNPrice

,AZN
2000-01-03,8.473514
2000-01-04,8.190209
2000-01-05,8.228842
2000-01-06,8.203084
2000-01-07,8.550780
...,...
2024-02-12,60.480690
2024-02-13,60.106438
2024-02-14,60.726906
2024-02-15,61.889050


In [28]:
MRKpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/MRKdata.csv'
MRKPrice = pd.read_csv(MRKpath)
MRKPrice = MRKPrice.set_index(pd.DatetimeIndex(MRKPrice['Date'].values))
MRKPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
MRKPrice = MRKPrice.rename(columns={"Adj Close": "MRK"})
MRKPrice

,MRK
2000-01-03,26.873911
2000-01-04,25.930075
2000-01-05,26.948402
2000-01-06,27.171942
2000-01-07,29.779860
...,...
2024-02-12,125.339996
2024-02-13,125.430000
2024-02-14,125.820000
2024-02-15,126.540001


In [29]:
RHHBYpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/RHHBYdata.csv'
RHHBYPrice = pd.read_csv(RHHBYpath)
RHHBYPrice = RHHBYPrice.set_index(pd.DatetimeIndex(RHHBYPrice['Date'].values))
RHHBYPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
RHHBYPrice = RHHBYPrice.rename(columns={"Adj Close": "RHHBY"})
RHHBYPrice

,RHHBY
2003-08-20,5.621511
2003-08-21,5.562044
2003-08-22,5.541054
2003-08-25,5.513070
2003-08-26,5.488582
...,...
2024-02-12,32.599998
2024-02-13,32.130001
2024-02-14,32.020000
2024-02-15,32.099998


In [30]:
ABBVpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/ABBVdata.csv'
ABBVPrice = pd.read_csv(ABBVpath)
ABBVPrice = ABBVPrice.set_index(pd.DatetimeIndex(ABBVPrice['Date'].values))
ABBVPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
ABBVPrice = ABBVPrice.rename(columns={"Adj Close": "ABBV"})
ABBVPrice

,ABBV
2013-01-02,22.207655
2013-01-03,22.024277
2013-01-04,21.746048
2013-01-07,21.790310
2013-01-08,21.316057
...,...
2024-02-12,173.070007
2024-02-13,173.289993
2024-02-14,174.419998
2024-02-15,176.589996


In [31]:
JNJpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/JNJdata.csv'
JNJPrice = pd.read_csv(JNJpath)
JNJPrice = JNJPrice.set_index(pd.DatetimeIndex(JNJPrice['Date'].values))
JNJPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
JNJPrice = JNJPrice.rename(columns={"Adj Close": "JNJ"})
JNJPrice

,JNJ
2000-01-03,24.698624
2000-01-04,23.794403
2000-01-05,24.045578
2000-01-06,24.799091
2000-01-07,25.854017
...,...
2024-02-12,156.660538
2024-02-13,155.290924
2024-02-14,154.566437
2024-02-15,156.729996


In [33]:
NVSpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/NVSdata.csv'
NVSPrice = pd.read_csv(NVSpath)
NVSPrice = NVSPrice.set_index(pd.DatetimeIndex(NVSPrice['Date'].values))
NVSPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
NVSPrice = NVSPrice.rename(columns={"Adj Close": "NVS"})
NVSPrice

,NVS
2000-01-03,15.844910
2000-01-04,15.545438
2000-01-05,15.613502
2000-01-06,15.953812
2000-01-07,16.198835
...,...
2024-02-12,100.529999
2024-02-13,99.059998
2024-02-14,99.050003
2024-02-15,99.629997


In [53]:
BMYpath = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/BMYdata.csv'
BMYPrice = pd.read_csv(BMYpath)
BMYPrice = BMYPrice.set_index(pd.DatetimeIndex(BMYPrice['Date'].values))
BMYPrice.drop(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
BMYPrice = BMYPrice.rename(columns={"Adj Close": "BMY"})
BMYPrice

,BMY
2000-01-03,25.552813
2000-01-04,23.619635
2000-01-05,23.468300
2000-01-06,23.791847
2000-01-07,27.275999
...,...
2024-02-12,49.750000
2024-02-13,48.709999
2024-02-14,48.889999
2024-02-15,49.439999


In [54]:
from pathlib import Path
import os

# Combine all the df and get a df file named assets. Work on assets
assets = pd.concat([PFEPrice, GSKPrice, AZNPrice, BMYPrice, MRKPrice, RHHBYPrice, SNYPrice, 
                    NVSPrice, ABBVPrice, JNJPrice], axis=1)
assets

,PFE,GSK,AZN,BMY,MRK,RHHBY,SNY,NVS,ABBV,JNJ
2000-01-03,13.061072,18.336874,8.473514,25.552813,26.873911,NaN,NaN,15.844910,NaN,24.698624
2000-01-04,12.574479,17.862465,8.190209,23.619635,25.930075,NaN,NaN,15.545438,NaN,23.794403
2000-01-05,12.779366,18.460629,8.228842,23.468300,26.948402,NaN,NaN,15.613502,NaN,24.045578
2000-01-06,13.240340,18.192493,8.203084,23.791847,27.171942,NaN,NaN,15.953812,NaN,24.799091
2000-01-07,14.136689,19.141298,8.550780,27.275999,29.779860,NaN,NaN,16.198835,NaN,25.854017
...,...,...,...,...,...,...,...,...,...,...
2024-02-12,27.660000,40.672421,60.480690,49.750000,125.339996,32.599998,46.340000,100.529999,173.070007,156.660538
2024-02-13,26.969999,40.989323,60.106438,48.709999,125.430000,32.130001,45.750000,99.059998,173.289993,155.290924
2024-02-14,27.110001,41.395355,60.726906,48.889999,125.820000,32.020000,45.799999,99.050003,174.419998,154.566437
2024-02-15,27.510000,41.365646,61.889050,49.439999,126.540001,32.099998,46.389999,99.629997,176.589996,156.729996


In [56]:
# save as a csv file
path = 'E:/Desktop/Extracurriculars/Projects & Competitions/DS Guidelight/Data/assets.csv'
Setpath = Path(path)
Setpath.parent.mkdir(parents=True, exist_ok=True)  
assets.to_csv(Setpath)

In [48]:
# Get the simple daily returns of PFE
daily_simple_returns = assets.pct_change()
daily_simple_returns

,PFE,GSK,AZN,BMY,MRK,RHHBY,SNY,NVS,ABBV,JNJ
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,-0.037255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,0.016294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,0.036072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,0.067698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-02-12,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006953
2024-02-13,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.008743
2024-02-14,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.004665
2024-02-15,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013998


In [57]:
# Calculate the annualized returns
annualized_returns = daily_simple_returns.mean() * 252
annualized_returns

PFE      0.006830
GSK      0.007374
AZN      0.016583
BMY      0.010451
MRK      0.019153
RHHBY    0.022519
SNY      0.024366
NVS      0.039594
ABBV     0.070206
JNJ      0.095226
dtype: float64

In [58]:
# Sort the data
sorted_annualized_returns = annualized_returns.sort_values(ascending=False)
sorted_annualized_returns

JNJ      0.095226
ABBV     0.070206
NVS      0.039594
SNY      0.024366
RHHBY    0.022519
MRK      0.019153
AZN      0.016583
BMY      0.010451
GSK      0.007374
PFE      0.006830
dtype: float64

#### Get asset_weights

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

asset_returns_orig = pd.read_csv('asset_returns.csv', index_col='Year', parse_dates=True)
asset_weights = pd.read_csv('asset_weights.csv', index_col='asset_class')
cols = ['Global Bonds (Unhedged)','Total US Bond Market','US Large Cap Growth',
            'US Large Cap Value','US Small Cap Growth','US Small Cap Value','Emerging Markets',
            'Intl Developed ex-US Market','Short Term Treasury']
asset_returns = asset_returns_orig[cols].dropna()
treasury_rate = asset_returns['Short Term Treasury']
asset_returns = asset_returns[cols[:-1]].astype(np.float).dropna()
asset_weights = asset_weights.loc[cols[:-1]]